In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
df_train=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
X_test=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
X_train=df_train.drop(["Class","Id","EJ"],inplace=False,axis=1)
Y_train=df_train["Class"]

In [3]:
impute=KNNImputer()
arr=impute.fit_transform(X_train)
X_train=pd.DataFrame(arr,columns=impute.feature_names_in_)

In [4]:
'''rfc=RandomForestClassifier()
rfc.fit(X_train,Y_train)
features_array=np.concatenate((np.array(X_train.columns).reshape(-1,1),rfc.feature_importances_.reshape(-1,1)),axis=1)
indices=np.argsort(features_array[:,-1])
most_imp_cols=list(features_array[indices[::-1]][:15,0])
most_imp_cols'''

'rfc=RandomForestClassifier()\nrfc.fit(X_train,Y_train)\nfeatures_array=np.concatenate((np.array(X_train.columns).reshape(-1,1),rfc.feature_importances_.reshape(-1,1)),axis=1)\nindices=np.argsort(features_array[:,-1])\nmost_imp_cols=list(features_array[indices[::-1]][:15,0])\nmost_imp_cols'

In [5]:
most_imp_cols=['DU',
 'FL',
 'GL',
 'DA',
 'BC',
 'CR',
 'DI',
 'DE',
 'EH',
 'AB',
 'AF',
 'FR',
 'FD ',
 'FE',
 'BQ']

In [6]:
rfc=RandomForestClassifier()
rfc.fit(X_train[most_imp_cols],Y_train)
predictions=rfc.predict_proba(X_test[most_imp_cols])
predictions

array([[0.59, 0.41],
       [0.59, 0.41],
       [0.59, 0.41],
       [0.59, 0.41],
       [0.59, 0.41]])

In [7]:
sub = X_test[['Id']].copy().copy()
sub['class_0'] = predictions[:,0]
sub['class_1'] = predictions[:,1]
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Id       5 non-null      object 
 1   class_0  5 non-null      float64
 2   class_1  5 non-null      float64
dtypes: float64(2), object(1)
memory usage: 248.0+ bytes


In [8]:
sub.to_csv('/kaggle/working/submission.csv', index=False)